<a href="https://colab.research.google.com/github/arinaaandreeva/Lab_MNIST/blob/main/Lab_MNIST_AndreevaAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №2
**Задание:**

Решить задачу классификации рукописных цифр на датасете MNIST https://www.kaggle.com/datasets/hojjatk/mnist-dataset.

#### Load data

In [ ]:
import gzip
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
import torch.optim as optim

In [ ]:
!git clone https://github.com/arinaaandreeva/Lab_MNIST

fatal: destination path 'Lab_MNIST' already exists and is not an empty directory.


In [ ]:
def load_mnist_images(file_path):
    with gzip.open(file_path, 'rb') as f:
        f.read(16) # пропускаем заголовки
        data = np.frombuffer(f.read(), dtype=np.uint8).reshape(-1, 28, 28)
    return data

def load_mnist_labels(file_path):
    with gzip.open(file_path, 'rb') as f:
        f.read(8)# пропускаем заголовки
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

images_path = '/content/Lab_MNIST/train-images-idx3-ubyte.gz'
labels_path = '/content/Lab_MNIST/train-labels-idx1-ubyte.gz'

# images = load_mnist_images(images_path)
# labels = load_mnist_labels(labels_path)

images_test = load_mnist_images('/content/Lab_MNIST/t10k-images-idx3-ubyte.gz')
labels_test = load_mnist_labels('/content/Lab_MNIST/t10k-labels-idx1-ubyte.gz')

# print(f"Images shape: {images.shape}")
# print(f"Labels shape: {labels.shape}")


In [ ]:
# Нормализация данных
def preprocess_images(images):
    # Преобразуем размерность (N, 28, 28) -> (N, 784)
    return images.reshape(images.shape[0], -1) / 255.0  # Нормализация в диапазон [0, 1]

def one_hot_encode(labels, num_classes=10):
    one_hot = np.zeros((labels.size, num_classes))
    one_hot[np.arange(labels.size), labels] = 1
    return one_hot


In [ ]:
images = preprocess_images(load_mnist_images(images_path))
labels = load_mnist_labels(labels_path)

#### Решение в виде нейронной сети, написанной на numpy

In [ ]:
class Linear_np:
    def __init__(self, in_features, out_features):
        self.W = np.random.randn(in_features, out_features) * 0.01  # Инициализация весов
        self.b = np.zeros((1, out_features))  # Инициализация смещений

    def forward(self, X):
        # Линейная трансформация Y = X*W+b
        self.X = X
        self.output = np.dot(X, self.W) + self.b
        return self.output

    def backward(self, dL_dout, learning_rate):
        # Вычисление градиентов
        self.dW = np.dot(self.X.T, dL_dout)  # Градиент по весам dL/dW
        self.db = np.sum(dL_dout, axis=0, keepdims=True)  # Градиент по смещениям dL/db
        dL_dX = np.dot(dL_dout, self.W.T)  # Градиент для входа dL/dX

        # Обновление параметров
        self.W -= learning_rate * self.dW
        self.b -= learning_rate * self.db

        return dL_dX

In [ ]:
class ReLU:

    def forward(self, X):
        self.X = X
        self.output = np.maximum(0, X)
        return self.output

    def backward(self, dL_dout):
        dL_dX = dL_dout * (self.X > 0).astype(float)  # Для обратного распространения вычисляем производную ReLU и умножаем на входно йградиент
        return dL_dX


class Softmax:
    def forward(self, X):
        exp_X = np.exp(X - np.max(X, axis=1, keepdims=True)) # Стабилизирует выяисление и предотвращает переполнение
        self.output = exp_X / np.sum(exp_X, axis=1, keepdims=True)
        return self.output

    def backward(self, dL_dout):
        # Для задачи классификации backward Softmax обычно комбинируется с функцией потерь
        return dL_dout  # Оставляем как есть


In [ ]:
class MSELoss:
    def forward(self, predictions, targets):
        self.predictions = predictions
        self.targets = targets
        loss = np.mean((predictions - targets) ** 2)
        return loss

    def backward(self):
      # Градиент по предсказаниям
        dL_dpred = 2 * (self.predictions - self.targets) / self.targets.shape[0]
        return dL_dpred


In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.fc1 = Linear_np(input_size, hidden_size)
        self.relu = ReLU()
        self.fc2 = Linear_np(hidden_size, output_size)
        self.softmax = Softmax()
        self.loss = MSELoss()

    def forward(self, X, y):
        # Данные проходят через линейный слой - RelU- линейный слой - softMax - MSELoss
        out = self.fc1.forward(X)
        out = self.relu.forward(out)
        out = self.fc2.forward(out)
        out = self.softmax.forward(out)
        loss = self.loss.forward(out, y)
        return loss, out

    def backward(self, learning_rate):
        # Потери подаются на вход обратно в сеть, градиенты обновляют параменты слоев
        dL_dout = self.loss.backward()
        dL_dout = self.softmax.backward(dL_dout)
        dL_dout = self.fc2.backward(dL_dout, learning_rate)
        dL_dout = self.relu.backward(dL_dout)
        self.fc1.backward(dL_dout, learning_rate)


In [ ]:
def train_model(X_train, y_train, X_test, y_test, epochs=30, learning_rate=0.1):
    input_size = X_train.shape[1]
    hidden_size = 256    # Размер скрытого слоя
    output_size = 10   # Классы

    model = NeuralNetwork(input_size, hidden_size, output_size)
    y_train_one_hot = one_hot_encode(y_train, output_size)
    for epoch in range(epochs):
        loss, predictions = model.forward(X_train, y_train_one_hot)
        model.backward(learning_rate)
        acc = np.mean(np.argmax(predictions, axis=1) == y_train)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}, Accuracy: {acc:.2f}")

    return model


In [ ]:
train_model(images, labels, images_test, labels_test)

Epoch 1/30, Loss: 0.0900, Accuracy: 0.09
Epoch 2/30, Loss: 0.0899, Accuracy: 0.20
Epoch 3/30, Loss: 0.0898, Accuracy: 0.34
Epoch 4/30, Loss: 0.0897, Accuracy: 0.43
Epoch 5/30, Loss: 0.0896, Accuracy: 0.48
Epoch 6/30, Loss: 0.0895, Accuracy: 0.51
Epoch 7/30, Loss: 0.0894, Accuracy: 0.54
Epoch 8/30, Loss: 0.0893, Accuracy: 0.55
Epoch 9/30, Loss: 0.0891, Accuracy: 0.56
Epoch 10/30, Loss: 0.0889, Accuracy: 0.57
Epoch 11/30, Loss: 0.0887, Accuracy: 0.57
Epoch 12/30, Loss: 0.0884, Accuracy: 0.58
Epoch 13/30, Loss: 0.0881, Accuracy: 0.58
Epoch 14/30, Loss: 0.0878, Accuracy: 0.58
Epoch 15/30, Loss: 0.0874, Accuracy: 0.58
Epoch 16/30, Loss: 0.0869, Accuracy: 0.59
Epoch 17/30, Loss: 0.0863, Accuracy: 0.59
Epoch 18/30, Loss: 0.0857, Accuracy: 0.60
Epoch 19/30, Loss: 0.0850, Accuracy: 0.61
Epoch 20/30, Loss: 0.0841, Accuracy: 0.62
Epoch 21/30, Loss: 0.0832, Accuracy: 0.64
Epoch 22/30, Loss: 0.0821, Accuracy: 0.65
Epoch 23/30, Loss: 0.0809, Accuracy: 0.67
Epoch 24/30, Loss: 0.0796, Accuracy: 0.68
E

#### Тесты методов forward и backward, что аутпуты этих методов совпадают с результатами в pytorch.

In [ ]:
# Numpy модель
np_input = images  # (batch_size, 784)
torch_input = torch.tensor(np_input, dtype=torch.float32, requires_grad=True)

input_size = 784
hidden_size = 256
output_size = 10
np_labels_onehot = np.eye(output_size)[labels]  # OneHot метки
np_nn = NeuralNetwork(input_size, hidden_size, output_size)
np_loss, np_output = np_nn.forward(np_input, np_labels_onehot)

# градиент для Numpy
np_nn.backward(learning_rate=0.1)

In [ ]:
# параметры
input_size = 784
hidden_size = 256
output_size = 10
learning_rate = 0.1

torch_input = torch.tensor(np_input, dtype=torch.float32, requires_grad=True)
np_labels_onehot = np.eye(output_size)[labels]  # OneHot метки
torch_labels_onehot = torch.tensor(np_labels_onehot, dtype=torch.float32)

torch_nn = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, output_size),
    nn.Softmax(dim=1)
)


# Оптимизатор для определения lr и loss
optimizer = optim.SGD(torch_nn.parameters(), lr=learning_rate)
criterion = nn.MSELoss()



# Пример вперед (forward pass)
torch_output = torch_nn(torch_input)

# Вычисление потерь
loss = criterion(torch_output, torch_labels_onehot)

# Обратный проход (backward pass)
optimizer.zero_grad()  # Обнуляем градиенты перед вычислением новых
loss.backward()        # Вычисляем градиенты
optimizer.step()       # Обновляем параметры модели с использованием learning rate


In [ ]:
# Сравнение выходных данных
print(f"Forward разница вывода: {np.mean(np.abs(np_output - torch_output.detach().numpy())):.5f}")


Forward разница вывода: 0.00673


In [ ]:
print(f"Градиенты для весов 1 слоя: {np.mean(np.abs(np_nn.fc1.dW - torch_nn[0].weight.grad.detach().numpy().T)):.5f}")
print(f"Градиенты для смещений 1 слоя: {np.mean(np.abs(np_nn.fc1.db -  - torch_nn[0].bias.grad.detach().numpy().T)):.5f}")
print(f"Градиенты для весов 2 слоя: {np.mean(np.abs(np_nn.fc2.dW -torch_nn[2].weight.grad.detach().numpy().T) ):.5f}")
print(f"Градиенты для смещений 2 слоя: {np.mean(np.abs(np_nn.fc2.db - torch_nn[2].bias.grad.detach().numpy().T)):.5f}")
print(f'Разница Loss {loss - np_loss:.5f}')

Градиенты для весов 1 слоя: 0.00024
Градиенты для смещений 1 слоя: 0.00077
Градиенты для весов 2 слоя: 0.00291
Градиенты для смещений 2 слоя: 0.00790
Разница Loss 0.00017


In [ ]:
# model.cuda(), with torch.cuda.amp.autocast
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model.to(device)

In [ ]:
# """Для forward"""
# np_input = images  # (batch_size, 784)
# torch_input = torch.tensor(np_input, dtype=torch.float32, requires_grad=True)

# input_size = 784
# hidden_size = 256
# output_size = 10

# # Numpy модель
# np_layer1 = Linear(input_size, hidden_size)
# np_layer2 = Linear(hidden_size, output_size)
# hidden_output_np = np_layer1.forward(np_input)
# final_output_np = np_layer2.forward(hidden_output_np)

# # PyTorch модель
# torch_layer1 = torch.nn.Linear(input_size, hidden_size)
# torch_layer2 = torch.nn.Linear(hidden_size, output_size)

# # Сопоставление параметров Numpy и PyTorch
# torch_layer1.weight.data = torch.tensor(np_layer1.W.T, dtype=torch.float32)
# torch_layer1.bias.data = torch.tensor(np_layer1.b.flatten(), dtype=torch.float32)
# #обновляем bias с использованием torch.no_grad(), Скопируем bias из numpy
# with torch.no_grad():
#     torch_layer1.bias.copy_(torch.tensor(np_layer1.b.flatten(), dtype=torch.float32))
# torch_layer2.weight.data = torch.tensor(np_layer2.W.T, dtype=torch.float32)
# torch_layer2.bias.data = torch.tensor(np_layer2.b.flatten(), dtype=torch.float32)

# torch_input = torch.tensor(np_input, dtype=torch.float32, requires_grad=True)  # PyTorch input
# hidden_output_torch = torch_layer1(torch_input)
# final_output_torch = torch_layer2(hidden_output_torch)

# # Вывод разницы между результатами
# abs_difference = np.abs(final_output_np - final_output_torch.detach().numpy())
# rel_difference = abs_difference
# print("разница между выходными данными:", np.mean(rel_difference))
# print("Относительная разница между выходными данными:", np.mean(rel_difference/ np.abs(final_output_np)))




In [ ]:
# # PyTorch модель
# torch_input = torch.tensor(np_input, dtype=torch.float32, requires_grad=True)
# torch_labels_onehot = torch.tensor(np_labels_onehot, dtype=torch.float32)

# torch_nn = nn.Sequential(
#     nn.Linear(input_size, hidden_size),
#     nn.ReLU(),
#     nn.Linear(hidden_size, output_size),
#     nn.Softmax(dim=1)
# )

# torch_output = torch_nn(torch_input)
# torch_loss = torch.mean((torch_output - torch_labels_onehot) ** 2)

# # Сравнение выходных данных
# print(f"Forward разница вывода: {np.mean(np.abs(np_output - torch_output.detach().numpy())):.5f}")
# print(f"Forward относительная разница вывода: {np.mean(np.abs((np_output - torch_output.detach().numpy())/np_output)):.5f}")

Правила следующие:
* нужно представить решение в виде нейронной сети, написанной на numpy, и обученной с помощью алгоритма градиентного спуска;
* нейронная сеть должна состоять из двух линейных слоев, активаций relu и softmax, и mse лосса - каждый оформлен в виде класса с методами forward и backward;
* нельзя пользоваться автоградиентом (pytorch, numpy), за исключением тестов. Градиенты должны считаться вручную по алгоритму обратного распространения ошибки,
используя аналитические формулы производных;
* решение считается валидным, если оно достигает аккураси больше 50%.
* для каждого слоя должны быть реализованы тесты методов forward и backward. Нужно убедиться, что аутпуты этих методов совпадают с результатами в pytorch.

Критерии оценивания:
+ базовая 6 баллов
+ за линейный forward + backward - +1 балл
+ за классы активации softmax и relu - +1 балл
+ за класс MSELoss - +1 балл (не CrossEntropy - так интереснее)
+ +1 балл, если реализованы тесты и есть численное совпадение с результатами pytorch
- Плохое оформление кода - нет классов и разделений на функции - -1 балл
- Слишком медленный код - -1 балл. (если код можно ускорить в 10 раз)
- CrossEntropy вместо MSELoss - -1 балл.

Дедлайн: 5 декабря 23:59